In [1]:
import pandas as pd
import numpy as np
import os
import gc
import re
from collections import defaultdict, Counter
from num2words import num2words
import pickle
import inflect
p = inflect.engine()
from datetime import datetime
path = "/Users/alvira/Desktop/ml1/kaggle/textnlp/"

In [20]:
def is_float(string):
    try:
        return float(string.replace(',','')) and "." in string # True if string is a number contains a dot
    except ValueError:  # String is not a number
        return False
    
def digit2num(key):
    try:
        p = num2words(int(key.replace(',','')))
        p = p.replace(' and','').replace('-',' ').replace(',','')
        return p.lower()
    except:
        return key

def float2num(key):
    key = float(key.replace(',',''))
    key = p.number_to_words(key,decimal='point',andword='')
#     if 'zero' == key.split()[-1]:
#         key = p[:-5]
    if 'zero' == key.split()[0]:
        key = key[5:]
    key = key.replace('-',' ').replace(',','')
    return key.lower()

from dateutil.parser import parse
def is_date(string):
    try: 
        parse(string)
        return True
    except ValueError:
        return False

def is_num(key):
    
    if is_float(key) or re.match(r'^-?[0-9]\d*?$', key.replace(',','')): return True
    else: return False

def num2word(key):
    if re.match(r'^-?\d+$', key.replace(',','')):
        return digit2num(key)
    if is_float(key):
        return float2num(key)
        

In [19]:
dict_m = {'"': 'inches', "'": 'feet', 'km/s': 'kilometers per second', 'AU': 'units', 'BAR': 'bars', 'CM': 'centimeters', 'mm': 'millimeters', 'FT': 'feet', 'G': 'grams', 
     'GAL': 'gallons', 'GB': 'gigabytes', 'GHZ': 'gigahertz', 'HA': 'hectares', 'HP': 'horsepower', 'HZ': 'hertz',
     'KA':'kilo amperes', 'KB': 'kilobytes', 'KG': 'kilograms', 'KHZ': 'kilohertz', 'KM':'kilometers', 'km3': 'cubic kilometers',
      'KM2': 'square kilometers', 'KM²': 'square kilometers', 'KT': 'knots', 'KV': 'kilo volts', 
     'KW': 'kilowatts', 'Kw':'kilowatts', 'KWH': 'kilo watt hours', 'LB': 'pounds', 'LBS': 'pounds', 'M': 'meters',
     'M2': 'square meters', 'M3': 'cubic meters', 'km/h': 'kilometers per hour','MA': 'mega amperes', 'MB': 'megabytes',
     'MB/S': 'megabytes per second', 'MG': 'milligrams', 'MHZ': 'megahertz', 'MI': 'miles', ''
     'ML': 'milliliters', 'MPH': 'miles per hour', 'MS': 'milliseconds', 'MV': 'milli volts', 'kJ':'kilojoules',
     'MW': 'megawatts', 'M²': 'square meters', 'M³': 'cubic meters', 'OZ': 'ounces', 'V': 'volts', 
     'YD': 'yards', 'au': 'units', 'bar': 'bars', 'cm': 'centimeters', 'ft': 'feet', 'g': 'grams', 
     'gal': 'gallons', 'gb': 'gigabytes', 'ghz': 'gigahertz', 'ha': 'hectares', 'hp': 'horsepower', 
     'hz': 'hertz', 'kWh': 'kilo watt hours', 'ka': 'kilo amperes', 'kb': 'kilobytes', 'kg': 'kilograms',
     'khz': 'kilohertz', 'km': 'kilometers', 'km2': 'square kilometers', 'km²': 'square kilometers', 'kt': 'knots',
     'kv': 'kilo volts', 'kw': 'kilowatts', 'lb': 'pounds', 'lbs': 'pounds', 'm': 'meters', 'm2': 'square meters',
     'm3': 'cubic meters', 'ma': 'mega amperes', 'mb': 'megabytes', 'mb/s': 'megabytes per second', 
     'mg': 'milligrams', 'mhz': 'megahertz', 'mi': 'miles', 'ml': 'milliliters', 'mph': 'miles per hour',
     'ms': 'milliseconds', 'mv': 'milli volts', 'mw': 'megawatts', 'm²': 'square meters', 'm³': 'cubic meters', 
     'oz': 'ounces', 'v': 'volts', 'yd': 'yards', 'µg': 'micrograms', 'ΜG': 'micrograms', 'kg/m3': 'kilograms per meter cube'}

dict_mon = {'jan': "January", "feb": "February", "mar ": "march", "apr": "april", "may": "may ",
            "jun": "june", "jul": "july", "aug": "august","sep": "september",
            "oct": "october","nov": "november","dec": "december", "january":"January", "february":"February",
            "march":"march", "april":"april", "may": "may", "june":"june","july":"july", "august":"august", 
            "september":"september", "october":"october", "november":"november", "december":"december"}
        
lis = ['.co','.org','http','.gov','www.','.net','.info']

In [90]:
# files = [ 'output_1.csv', 'output_6.csv', 'output_11.csv', 'output_16.csv', 'output_21.csv','output_91.csv','output_96.csv']
# d1 = defaultdict(list)
# for file in files:
#     print(file)
#     train = pd.read_csv('/Users/alvira/Desktop/ml1/kaggle/textnlp/extraData/'+file,encoding='iso-8859-1' )
#     for i in range(train.shape[0]):
#         if train.iloc[i,1] == '<self>' or train.iloc[i,1] == 'sil':
#             train.iloc[i,1] = train.iloc[i,0]
#     train_list = [(train.iloc[i,0],train.iloc[i,1]) for i in range(train.shape[0])]
#     print("appending")
#     for k,v in train_list:
#         d1[k].append(v)
# counter_dict2 = {}
# print('dictionay')
# for key in d1:
#     c = Counter(d1[key]).most_common(1)[0][0]
#     counter_dict2[key] = c

In [17]:
train = pd.read_csv(path+'en_train.csv')
test = pd.read_csv(path+'en_test.csv')
d = defaultdict(list)
train_list = [(train.iloc[i,3],train.iloc[i,4]) for i in range(train.shape[0])]
for k,v in train_list:
    d[k].append(v)
counter_dict = {}
for key in d:
    c = Counter(d[key]).most_common(1)[0][0]
    counter_dict[key] = c
with open(path+'train_dict', "rb") as f:
    pickle.dump(counter_dict, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
data = []
Headers = ['id','after']
for i in range(test.shape[0]):
    templist =[]
    key = test.iloc[i,2]
    templist.append(str(test.iloc[i,0])+'_'+str(test.iloc[i,1]))
    if key in counter_dict:
        templist.append(counter_dict[test.iloc[i,2]])
    
        
    elif key in counter_dict2:
#         print('case 12: ' + key), 
        templist.append(counter_dict2[test.iloc[i,2]])
#         if key != counter_dict2[key]: 
#             print key , ':', counter_dict2[test.iloc[i,2]]
#         print(counter_dict2[test.iloc[i,2]])
    
    elif any(i in key for i in lis):
#         print('case 10: ' + key),
        key = key.replace('.',' dot ').replace('/',' slash ').replace('-',' dash ').replace(':',' colon ').replace('_',' underscore ')
        key = key.split()
        lis2 = ['dot','slash','dash','colon']
        for i in range(len(key)):
            if key[i] not in lis2:
                key[i]=" ".join(key[i])
        key = " ".join(key)
        templist.append(key.lower())
    #         print(key.lower())
    
    elif key in counter_dict3:
        templist.append(counter_dict3[test.iloc[i,2]])
        print(key+ ': '+ counter_dict3[test.iloc[i,2]] )
        
        
    elif is_num(key.strip()):
        if len(key.replace(',',''))>9 and key.isdigit(): 
            text = p.number_to_words(key, zero = 'o', group = 1).replace(',','')
#             print key + ': '+ text
        else: 
            text = num2word(key.strip())
            if key.startswith('0') and '.' not in key: 
                text = p.number_to_words(key, zero = 'o', group = 1).replace(',','')
#                 print key + ':' + text
        templist.append(text.lower())
#             print(num2word(key))

    elif key.split()[-1] in dict_m:
#          print('case 2: ' + key)
        unit = dict_m[key.split()[-1]]
        val = key.split()[0]
        if is_num(val):
            val = num2word(val)
            text = val + ' ' + unit
        else: text = key
        templist.append(text)
#             print(text)

    elif '/km2' in key or '/km²' in key:
#             print('case 7: ' + key),
        key = key.replace('/km2','').replace('/km²','')
        if is_num(key):
            text = num2word(key) + ' '+'per square kilometers'
        else: text = key
        templist.append(text)
#             print(text)

    elif key == "inf": templist.append("inf")

    elif key[-1] == '%': 
        val = key[:-1]
        if is_num(val):
#                 print('case 3.1: ' + key),
            val = num2word(val)
            text = val + ' ' + 'percent'
        else: text = key
        templist.append(text)
#             print(text)

    
    elif re.match('[0-9]*:[0-9]',key): 
        v = key.split()
        if len(v) == 1:
            v[0] = v[0].split(':')
            try:
                text = ' '.join([digit2num(i) for i in v[0]])
            except:
                text = key
        elif len(v) == 2:
            v[0] = v[0].split(':')
            v[1] = v[1].replace('.','')
            v[1] = " ".join(v[1])
            text = ' '.join([digit2num(i) for i in v[0]]) + ' ' + v[1]
        else: text = key
        templist.append(text.lower())
#         print key + '-' + text.lower() 
            
    elif is_date(key):
#         print key
        v = key.split('-')
        if len(v)==3:
#             print('case 4.1: ' + key)
            if v[1].isdigit():

                try:
                    date = datetime.strptime(key , '%Y-%m-%d')
                    text = 'the '+ p.ordinal(p.number_to_words(int(v[2]))).replace('-',' ')+' of '+datetime.date(date).strftime('%B')
                    if int(v[0])>=2000 and int(v[0]) < 2010:
                        text = text  + ' '+digit2num(v[0])
                    else: 
                        text = text + ' ' + digit2num(v[0][0:2]) + ' ' + digit2num(v[0][2:])
                except:
                     text = key
                templist.append(text.lower())
#                 print "case a: "+text.lower()
#                     print(text.lower())
        else:   
            v = re.sub(r'[^\w]', ' ', key).split()
            if v[0].isalpha():
#                 print('case 4.2.1: ' + key),
                try:
                    if len(v)==3:
                        text = dict_mon[v[0].lower()] + ' '+ p.ordinal(p.number_to_words(int(v[1]))).replace('-',' ')
                        if int(v[2])>=2000 and int(v[2]) < 2010:
                            text = text  + ' '+digit2num(v[2])
                        else: 
                            text = text + ' ' + digit2num(v[2][0:2]) + ' ' + digit2num(v[2][2:])   
                    elif len(v)==2:

                        if int(v[1])>=2000 and int(v[1]) < 2010:
                            text = dict_mon[v[0].lower()]  + ' '+ digit2num(v[1])
                        else: 
                            if len(v[1]) <=2:
                                text = dict_mon[v[0].lower()] + ' ' + digit2num(v[1])
                            else:
                                text = dict_mon[v[0].lower()] + ' ' + digit2num(v[1][0:2]) + ' ' + digit2num(v[1][2:])
                    else: text = key
                except:
                    text = key
                templist.append(text.lower())
#                 print "case b:" + text.lower()
#                     print(text.lower())

            else: 
                key = re.sub(r'[^\w]', ' ', key)
                v = key.split()
                try:
                    date = datetime.strptime(key , '%d %b %Y')
                    text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+ dict_mon[v[1].lower()]
                    if int(v[2])>=2000 and int(v[2]) < 2010:
                        text = text  + ' '+digit2num(v[2])
                    else: 
                        text = text + ' ' + digit2num(v[2][0:2]) + ' ' + digit2num(v[2][2:])
                except:
                    try:
                        date = datetime.strptime(key , '%d %B %Y')
                        text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+ dict_mon[v[1].lower()]
                        if int(v[2])>=2000 and int(v[2]) < 2010:
                            text = text  + ' '+digit2num(v[2])
                        else: 
                            text = text + ' ' + digit2num(v[2][0:2]) + ' ' + digit2num(v[2][2:])
                    except:
                        try:
                            date = datetime.strptime(key , '%d %m %Y')
                            text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+datetime.date(date).strftime('%B')
                            if int(v[2])>=2000 and int(v[2]) < 2010:
                                text = text  + ' '+digit2num(v[2])
                            else: 
                                text = text + ' ' + digit2num(v[2][0:2]) + ' ' + digit2num(v[2][2:])
                        except:
                            try:
                                date = datetime.strptime(key , '%d %m %y')
                                text = 'the '+ p.ordinal(p.number_to_words(int(v[0]))).replace('-',' ')+' of '+datetime.date(date).strftime('%B')
                                v[2] = datetime.date(date).strftime('%Y')
                                if int(v[2])>=2000 and int(v[2]) < 2010:
                                    text = text  + ' '+digit2num(v[2])
                                else: 
                                    text = text + ' ' + digit2num(v[2][0:2]) + ' ' + digit2num(v[2][2:])
                            except:
                                text = key

#                     print('case 4.2.2: ' + key),
                templist.append(text.lower())
#                 print "case c:" + text.lower()
#                     print(text.lower())

    elif re.match('^\$', key):
#         print key
        v = key.replace('$','').replace('US$','').split()
        if len(v) == 2: 
#                 print 'case 5.1: ' + key,
            if is_num(v[0]):
                text = num2word(v[0])
                key = text+ ' '+ v[1] + ' '+ 'dollars'

            templist.append(key.lower())
#                 print key.lower()
        elif is_num(v[0]):
#                 print 'case 5.2: ' + key,
            text = num2word(v[0])
            key = text + ' '+ 'dollars'
            templist.append(key.lower())
#                 print key.lower()
        else:
            if 'm' in key or 'M' in key or 'million':
                key = p.number_to_words(key).replace(',','').replace('-',' ').replace(' and','') + ' million dollars'
            elif 'bn' in key:
                key = p.number_to_words(key).replace(',','').replace('-',' ').replace(' and','') + ' billion dollars'
            templist.append(key.lower())
#             print key

    elif re.match('^\£', key):
#         print key
        key =  key.replace('£','')
        v = key.split()
        if len(v) == 2: 
#                 print 'case 5.1: ' + key,
            if is_num(v[0]):
                text = num2word(v[0])
                key = text+ ' '+ v[1] + ' '+ 'pounds'

            templist.append(key.lower())
#             print key.lower()
        elif is_num(v[0]):
#                 print 'case 5.2: ' + key,
            text = num2word(v[0])
            key = text + ' '+ 'pounds'
            templist.append(key.lower())
#             print key.lower()
        else:
            if 'm' in key or 'M' in key or 'million':
                key = p.number_to_words(key).replace(',','').replace('-',' ').replace(' and','') + ' million pounds'
            elif 'bn' in key:
                key = p.number_to_words(key).replace(',','').replace('-',' ').replace(' and','') + ' billion pounds'
            templist.append(key.lower())
#             print key.lower()

    elif re.match(r'^[a-zA-Z]\.', key): 
#             print 'case 8: ' + key,
        key = key.replace('.',' ')
        templist.append(key.replace('  ',' ').lower())
#             print key.lower()

    elif re.match(r'^([0-9\(\)\+ \-]*)$', key):
#         print('case 9: ' + key)
        key = key.replace('-','.').replace(')','.')
        key = p.number_to_words(key,group =1, decimal = "sil",zero = 'o').replace(',','')
        templist.append(key.lower())
#         print(key.lower())

    elif re.match('[0-9]*/[0-9]',key): 
        v = key.split('/')
        try:
            fraction = p.ordinal(p.number_to_words(v[1])) + 's'
            text = num2word(v[0]) + ' '+ fraction.replace('-',' ').replace(' and', '')
        except:
            text = key
        templist.append(text.lower())
#         print key, ':', text.lower()

    elif 'illion' in key:
        v = key.split()
        if len(v) ==2 and is_num(v[0]):
            text = num2word(v[0]) + ' ' + v[1].lower()
        else: text = key
        templist.append(text)
#         print key + ': '+ text

    elif key.isupper() and '-' in key and key.replace('-','').isalpha(): 
        text = ' '.join(key.replace('-','')).lower()
        templist.append(text)
#         print key, ':', text.lower()    
    
    else: 
#         print('case 6: ' + key),
#         if key.isalpha() == False and "'s" not in key and "s'" not in key: print key
#         if 'BC' in key or 'AD' in key and key.isalpha() == False : print key
        templist.append(test.iloc[i,2])
#         print(test.iloc[i,2])
    data.append(templist)

In [26]:
df = pd.DataFrame(data,columns=Headers)
df.to_csv(path+'Submission2.csv',index = False)

In [ ]:
files = os.listdir(path+'extraData/en_with_types/')

d1 = defaultdict(list)
for file in files:
    try:
        print(file)
        train = pd.read_csv(path+'extraData/en_with_types/'+file, sep='\t', names = ["c1","c2","c3"], \
                           error_bad_lines=False, quoting=3)

        del train["c1"]
        train2 = train[train['c3'] != "<self>"]
        train3 = train2[train2['c2'] != "<eos>"]
#         train3.drop_duplicates(inplace=True)
        train3 = train3[train3.c2.str.contains("\tsil") == False]
        train3.loc[train2['c3']=='sil','c3'] = train3['c2']

        train_list = [(train3.iloc[i,0],train3.iloc[i,1]) for i in range(train3.shape[0])]
        for k,v in train_list:
            d1[k].append(v)
    except: print("issue: "+file)

In [ ]:
counter_dict3 = {}
# print('dictionay')
for key in d1:
    c = Counter(d1[key]).most_common(1)[0][0]
    counter_dict3[key] = c

In [4]:
with open(path+'extra_dict3', "rb") as f:
    pickle.dump(counter_dict3, f, pickle.HIGHEST_PROTOCOL)

dictionay


In [ ]:
with open(path+'extra_dict2', "rb") as f:
    counter_dict3 = pickle.load(f)

In [ ]:
with open(path+'extra_dict', "rb") as f:
    counter_dict2 = pickle.load(f)

In [ ]:
for i in range(test.shape[0]):
    key = test.iloc[i,2]
    if key not in counter_dict:
        if key not in counter_dict2:
            if key in counter_dict3:
                print(key)

In [9]:
files = os.listdir(path+'extraData/en_with_types/')
file = files[1]
d1 = defaultdict(list)
# for file in files:
# try:
print(file)
train = pd.read_csv(path+'extraData/en_with_types/'+file, sep='\t', names = ["c1","c2","c3"], \
                    error_bad_lines=False, quoting=3)

del train["c1"]
train2 = train[train['c3'] != "<self>"]
train3 = train2[train2['c2'] != "<eos>"]
# train3.drop_duplicates(inplace=True)
train3 = train3[train3.c2.str.contains("\tsil") == False]
train3.loc[train2['c3']=='sil','c3'] = train3['c2']

train_list = [(train3.iloc[i,0],train3.iloc[i,1]) for i in range(train3.shape[0])]
for k,v in train_list:
    d1[k].append(v)
# except: print("issue: "+file)

output-00001-of-00100
